In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-seg.yaml")  # build a new model from YAML
model = YOLO("yolo11n-seg.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n-seg.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="/home/m4rc/Desktop/vc_final_assignament/Dataset/Cityscapes.v10i.yolov11/data.yaml", epochs=200, imgsz=640, patience=10)

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO('/home/m4rc/Desktop/vc_final_assignament/runs/segment/train3/weights/best.pt')

video_path = '/home/m4rc/Desktop/vc_final_assignament/video_demo.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error al abrir el video: {video_path}")
    exit()

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter('resultado_segmentacion.mp4', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Realiza la inferencia (image-size por defecto = 640 o lo que tenga el modelo)
    # Ajusta conf según tus necesidades
    results = model.predict(frame, conf=0.25)
    
    seg_result = results[0]
    seg_masks = seg_result.masks  # Instancias de segmentación

    if seg_masks is not None:
        for mask_tensor in seg_masks.data:
            # 1) Convertimos el tensor a numpy
            mask = mask_tensor.cpu().numpy()
            # mask tiene forma (redimensionada_altura, redimensionada_ancho)

            # 2) Redimensionamos la máscara al tamaño original del frame
            mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]),
                                      interpolation=cv2.INTER_NEAREST)

            # 3) Convertimos a booleano
            mask_bool = mask_resized.astype(bool)

            # 4) Definimos color y transparencia
            color = (0, 255, 0)
            alpha = 0.4

            # 5) Sobreponemos la máscara con alpha blending
            frame[mask_bool] = frame[mask_bool] * (1 - alpha) + np.array(color) * alpha

    cv2.imshow('Segmentacion YOLOv8', frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
